<a href="https://colab.research.google.com/github/FabioCerqueiraGit/DataScienceR/blob/main/DataScienceR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Vendas com R
Realizando análise de vendas com R e bibliotecas:


*   Tidyverse
*   Readxl



In [2]:
# Configurando o ambiente para usar R
%load_ext rpy2.ipython


# Instalando as bibliotecas Tidyverse e Readxl

In [ ]:
%%R
# Instalando e carregando os pacotes necessários
install.packages("tidyverse")
install.packages("readxl")
library(tidyverse)
library(readxl)

In [ ]:
# Carregar os dados
metas <- read_excel("https://github.com/FabioCerqueiraGit/DadosVendasDescaracterizados/blob/main/Metas.xlsx?raw=true")
produtos <- read_excel("https://github.com/FabioCerqueiraGit/DadosVendasDescaracterizados/blob/main/Produtos.xlsx?raw=true")
vendas <- read_excel("https://github.com/FabioCerqueiraGit/DadosVendasDescaracterizados/blob/main/Vendas.xlsx?raw=true")
vendedores <- read_excel("https://github.com/FabioCerqueiraGit/DadosVendasDescaracterizados/blob/main/Vendedores.xlsx?raw=true")


In [ ]:
# Mapeamento dos meses em português para números
meses <- c("janeiro" = 1, "fevereiro" = 2, "março" = 3, "abril" = 4, "maio" = 5, "junho" = 6,
           "julho" = 7, "agosto" = 8, "setembro" = 9, "outubro" = 10, "novembro" = 11, "dezembro" = 12)

# Criando a coluna MesNumero
metas <- metas %>%
  mutate(MesNumero = meses[tolower(Mês)],
         Trimestre = (MesNumero - 1) %/% 3 + 1,
         Semestre = (MesNumero - 1) %/% 6 + 1)

In [ ]:
# Removendo o prefixo "Prod " e convertendo para inteiro na coluna "Cod Produto"
produtos <- produtos %>%
  mutate(`Cod Produto` = as.integer(str_replace(`Cod Produto`, "Prod ", "")))

In [ ]:
# Unindo vendas com produtos
vendas_produtos <- vendas %>%
  inner_join(produtos, by = c("cdProduto" = "Cod Produto"))

# Unindo vendas com vendedores
vendas_completas <- vendas_produtos %>%
  inner_join(vendedores, by = c("cdVendedor" = "cdVendedor"))

In [ ]:
# Calculando o valor total das vendas
vendas_completas <- vendas_completas %>%
  mutate(ValorTotalVenda = PrecoUnitario * QtdItens)

# Imprimindo a soma total de ValorTotalVenda
soma_total_vendas <- sum(vendas_completas$ValorTotalVenda)
print(paste("Soma total de ValorTotalVenda:", soma_total_vendas))

In [ ]:
# Unindo vendas com metas
vendas_metas <- vendas_completas %>%
  inner_join(metas, by = c("cdProduto" = "cdProduto", "cdVendedor" = "Vendedor", "MesNumero" = "MesNumero")) %>%
  mutate(DifMeta = ValorTotalVenda - `Meta Faturamento`)

# Imprimindo a soma total de Meta Faturamento
soma_total_meta <- sum(vendas_metas$`Meta Faturamento`)
print(paste("Soma total de Meta Faturamento:", soma_total_meta))

In [ ]:
# Opções de filtro
opcao <- readline(prompt = "Escolha o filtro (mês, trimestre, semestre, ano): ")
opcao <- tolower(opcao)

if (opcao == "mês") {
  mes <- as.integer(readline(prompt = "Digite o número do mês (1-12): "))
  vendas_metas <- vendas_metas %>% filter(MesNumero == mes)
} else if (opcao == "trimestre") {
  trimestre <- as.integer(readline(prompt = "Digite o número do trimestre (1-4): "))
  vendas_metas <- vendas_metas %>% filter(Trimestre == trimestre)
} else if (opcao == "semestre") {
  semestre <- as.integer(readline(prompt = "Digite o número do semestre (1-2): "))
  vendas_metas <- vendas_metas %>% filter(Semestre == semestre)
} else if (opcao == "ano") {
  ano <- as.integer(readline(prompt = "Digite o ano: "))
  vendas_metas <- vendas_metas %>% filter(Ano == ano)
} else {
  print("Opção inválida. Gerando gráfico para todos os dados.")
}

In [ ]:
# Gráfico da quantidade de produtos vendidos
quantidade_produtos_vendidos <- vendas_metas %>%
  group_by(LinhaProduto) %>%
  summarise(QuantidadeVendida = sum(QtdItens))

ggplot(quantidade_produtos_vendidos, aes(x = LinhaProduto, y = QuantidadeVendida, fill = LinhaProduto)) +
  geom_bar(stat = "identity") +
  labs(title = "Quantidade de Produtos Vendidos", x = "Produto", y = "Quantidade Vendida") +
  theme_minimal() +
  theme(legend.position = "none")

In [ ]:
# Gráfico do total de vendas por vendedor
total_vendas_vendedores <- vendas_metas %>%
  group_by(Vendedor) %>%
  summarise(TotalVendas = sum(ValorTotalVenda))

ggplot(total_vendas_vendedores, aes(x = reorder(Vendedor, -TotalVendas), y = TotalVendas, fill = Vendedor)) +
  geom_bar(stat = "identity") +
  labs(title = "Total de Vendas por Vendedor", x = "Vendedor", y = "Total de Vendas") +
  theme_minimal() +
  theme(legend.position = "none") +
  coord_flip()  # Flip para facilitar a leitura dos nomes dos vendedores